In [7]:
from pathlib import Path
from IPython.display import HTML, display
import ipywidgets as widgets
import datetime
import pandas as pd
import numpy as np
base_path = "C:\\Users\\pa\\OneDrive\\7.UniCode\\0thesis-code\\runs"
#img_path = Path.cwd() / "img.jpg"
# display(Image(filename=img_path))
p = Path(base_path)
p.as_uri()

'file:///C:/Users/pa/OneDrive/7.UniCode/0thesis-code/runs'

In [8]:
test = widgets.Dropdown(
       options=["k8s_taewa_3lite_","asg_taewa_3lite_", "k8s_apache_3_", "k8s_taewa_3_", "asg_apache_3_", "asg_taewa_3_"],
       value='k8s_apache_3_',
       description='Test:')
all = widgets.Checkbox(value=True,
       description='All')
graphs=["QPS_estimatedProcessedBytes.png","QPS_cpuUtilization.png","QPS_groupInServiceCapacity.png"]
date_filter = widgets.Text(
       value='2022.0[3-9]', # use glob pattern http://pymotw.com/2/glob/
       description='Filter', )
box = widgets.HBox([date_filter, all, test ])
display(box)

In [9]:
# #tst=test.value
# nw = datetime.datetime.now()
# search=f'**/*{test.value}*/**/QPS_estim*.png'
# header="<style>table, th, td {border: 1px solid black;}</style>"
# header+=f"date: {nw}<br><br><table>"
# footer="</table>"
# iList=[]
# for i in p.glob(search):
#     dirr=i.parent.parent.name
#     img=i.as_uri()
#     line = f"<tr><td rowspan='1'>{dirr}</td></tr> \
#         <tr><td><img alt={dirr} src={img} /></td></tr>"
#     #display(Image(filename=i) )
#     iList.append(line)
# #print(imagesList)
# imagesList = ''.join(iList)
# page= header + imagesList
# a = HTML(page)
# html_src = a.data
# with open('html_file.html', 'w') as f:
#     f.write(html_src)
# #display(a, metadata=dict(isolated=True))

In [10]:
print(test._options_values)
test_array=[]
if all.value:
    test_array=test._options_values;
else:
    test_array.append(test.value)

for tst in test_array:
    search=f'*{date_filter.value}*{tst}*'
    nw = datetime.datetime.now()
    html_path=p/f'o_{tst}.html'
    header=f"<head><title>{tst}</title>"
    header+="<style>table, th, td {border: 1px solid black;} body {font: 16px Arial, sans-serif;}</style></head>"
    header+=f"date: {nw}<br><h2><a href='{html_path.as_uri()}'>{tst}</a></h2><br>"
    for h in p.glob('*.html'):
        if h.name.find(f'{tst}')>0:
            header+=">"
        header+=f"<a href='{h.as_uri()}'>{h.name}</a>&nbsp;|&nbsp;"
    header+="<table>"
    footer="</table>"
    iList=[]
    count=0
    for i in p.glob(search):
        count+=1
        dir = i.name
        if not (i/"csv").exists():
            line = f"<tr><td rowspan='1'><b>skip skip skip: {dir}</b></td></tr>"
            iList.append(line)
            continue
        ##================ get json Fortio data ================
        #print(i)
        json_paths = (i/"csv").glob('*.json')
        fortio_path=list(json_paths)[0]

        df = pd.read_json(fortio_path, lines=True)
        df['StartTime'] = pd.to_datetime(df['StartTime'])
        df.set_index(['StartTime'],inplace=True)

        #qps= df[["ActualQPS","NumThreads"]]
        #mydf = df[df['Labels'] == 'k8s_apache_3-performance-1']

        # get performance QPS
        perf_df = df[df['Labels'].str.contains('performance-[1-3]', regex=True)]
        perf_data=perf_df.ActualQPS

        # get scaling start times
        scaling_starts_df = df[df['Labels'].str.contains('scaling-.-1$', regex=True)]
        #print(df)
        #scaling_data=scaling_df.ActualQPS
        #print(scaling_df['Labels'])
        #print(perf_times.iloc[0])

        ## ================ PERFORMANCE CloudWatch data ================
        aws_metric1="cpuUtilization"
        csv_paths = (i/"csv").glob('*'+aws_metric1+'.csv')
        c_path=list(csv_paths)[0]
        perfomance_cpu_list=[]
        #print(c_path)
        for index,value in perf_df.iterrows():
            perf_start=index#pd.to_datetime(stime)
            #print(perf_start)
            mdf = pd.read_csv(c_path, parse_dates=['datetime'], index_col="datetime")
            # get 4 max:
            # mmdf=mdf[perf_start:perf_start+ pd.Timedelta(5, "m")]
            # mmmdf=mmdf.nlargest(4, aws_metric1).sort_index()
            # get 3 mid:
            mmmdf=mdf[perf_start+ pd.Timedelta(1, "m"):perf_start + pd.Timedelta(4, "m")]
            cpu_list=list(mmmdf[aws_metric1])
            average_cpu=np.mean(cpu_list)
            perfomance_cpu_list.append(average_cpu)
            #print(list(mmmdf.cpuUtilization))
        avg_cpu=np.mean(perfomance_cpu_list)

        ## ================ SCALING CloudWatch data ================
        aws_metric2="groupInServiceCapacity"
        scaling_durations=[]
        csv_paths2 = (i/"csv").glob('*'+aws_metric2+'.csv')
        c_path2=list(csv_paths2)[0]
        #print(c_path2)
        for index,value in scaling_starts_df.iterrows():
            scale_start=index
            #print(scale_start)
            cwdf2 = pd.read_csv(c_path2, parse_dates=['datetime'], index_col="datetime")
            ccwdf2=cwdf2[scale_start:scale_start+ pd.Timedelta(14, "m")]
            max_scaled=ccwdf2[ccwdf2[aws_metric2].ge(3)] #.index[0]

            print(ccwdf2.head(6))
            print(max_scaled.head(2))

            if max_scaled.size > 0:
                cw_reach_max_time=max_scaled.index[0]
                duration = cw_reach_max_time - index
                duration_in_s = duration.total_seconds()
                #minutes = divmod(duration_in_s, 60)[0]
                #print(reach_max_time)
                #s = scaling_starts_df.index.get_loc(cw_reach_max_time, method='nearest')
                #max_minute=scaling_starts_df[reach_max_time]
                
                # print(index)
                # print(cw_reach_max_time)
                # print(duration_in_s)
            else:
                duration_in_s = 0
            scaling_durations.append(duration_in_s)

        ##================ print out ================
        line = ""
        line = f"<tr><td rowspan='1'>{count}:  <a href='{i.as_uri()}'>{dir}</a> </td></tr><tr><td>column1</td><td>column2</td><td>column3</td><td>QPS</td><td>CPU%</td><td>Duration,s</td></tr><tr>"
        ####====== Images
        for y in graphs:
            #print(i.name)
            img=f'{i.name}/csv/{y}'
            line+=f"<td><img alt='{y}' src='{img}' /></td>"
        ####====== Data
        lst=list(map(int,perf_data))
        data=str(lst)[1:-1].replace(", ", "<br />")
        data_scaling=scaling_durations
        line+=f"<td>{data}</td>"
        line+=f"<td>{perfomance_cpu_list, avg_cpu}</td>" #[f'{aws_metric1}']
        line+=f"<td>{data_scaling}</td>"
        line+="</tr>"
        #line+="</tr><tr><td rowspan='1'>~~~</td></tr>"
        iList.append(line)
    imagesList = ''.join(iList)
    ##================ save HTML ================

    page= header + imagesList
    a = HTML(page)
    html_src = a.data
    with open(html_path, 'w') as f:
        f.write(html_src)
    display(a, metadata=dict(isolated=True))

('k8s_taewa_3lite_', 'asg_taewa_3lite_', 'k8s_apache_3_', 'k8s_taewa_3_', 'asg_apache_3_', 'asg_taewa_3_')
                           groupInServiceCapacity
datetime                                         
2022-03-26 09:31:00+00:00                       1
2022-03-26 09:32:00+00:00                       2
2022-03-26 09:33:00+00:00                       2
2022-03-26 09:34:00+00:00                       2
2022-03-26 09:35:00+00:00                       3
2022-03-26 09:36:00+00:00                       3
                           groupInServiceCapacity
datetime                                         
2022-03-26 09:35:00+00:00                       3
2022-03-26 09:36:00+00:00                       3
                           groupInServiceCapacity
datetime                                         
2022-03-26 09:49:00+00:00                       1
2022-03-26 09:50:00+00:00                       1
2022-03-26 09:51:00+00:00                       2
2022-03-26 09:52:00+00:00                  

                           groupInServiceCapacity
datetime                                         
2022-03-26 11:37:00+00:00                       1
2022-03-26 11:38:00+00:00                       1
2022-03-26 11:39:00+00:00                       1
2022-03-26 11:40:00+00:00                       1
2022-03-26 11:41:00+00:00                       1
2022-03-26 11:42:00+00:00                       2
                           groupInServiceCapacity
datetime                                         
2022-03-26 11:48:00+00:00                       3
2022-03-26 11:49:00+00:00                       3
                           groupInServiceCapacity
datetime                                         
2022-03-26 11:55:00+00:00                       2
2022-03-26 11:56:00+00:00                       2
2022-03-26 11:57:00+00:00                       2
2022-03-26 11:58:00+00:00                       2
2022-03-26 11:59:00+00:00                       2
2022-03-26 12:00:00+00:00                       2


                           groupInServiceCapacity
datetime                                         
2022-03-16 01:08:00+00:00                       1
2022-03-16 01:09:00+00:00                       2
2022-03-16 01:10:00+00:00                       2
2022-03-16 01:11:00+00:00                       2
2022-03-16 01:12:00+00:00                       2
2022-03-16 01:13:00+00:00                       3
                           groupInServiceCapacity
datetime                                         
2022-03-16 01:13:00+00:00                       3
2022-03-16 01:14:00+00:00                       3
                           groupInServiceCapacity
datetime                                         
2022-03-16 01:27:00+00:00                       2
2022-03-16 01:28:00+00:00                       2
2022-03-16 01:29:00+00:00                       2
2022-03-16 01:30:00+00:00                       3
2022-03-16 01:31:00+00:00                       3
2022-03-16 01:32:00+00:00                       3


                           groupInServiceCapacity
datetime                                         
2022-03-16 08:08:00+00:00                       2
2022-03-16 08:09:00+00:00                       2
2022-03-16 08:10:00+00:00                       2
2022-03-16 08:11:00+00:00                       3
2022-03-16 08:12:00+00:00                       3
2022-03-16 08:13:00+00:00                       3
                           groupInServiceCapacity
datetime                                         
2022-03-16 08:11:00+00:00                       3
2022-03-16 08:12:00+00:00                       3
                           groupInServiceCapacity
datetime                                         
2022-03-16 08:27:00+00:00                       1
2022-03-16 08:28:00+00:00                       2
2022-03-16 08:29:00+00:00                       2
2022-03-16 08:30:00+00:00                       2
2022-03-16 08:31:00+00:00                       3
2022-03-16 08:32:00+00:00                       3


                           groupInServiceCapacity
datetime                                         
2022-03-16 04:28:00+00:00                       1
2022-03-16 04:29:00+00:00                       1
2022-03-16 04:30:00+00:00                       1
2022-03-16 04:31:00+00:00                       1
2022-03-16 04:32:00+00:00                       2
2022-03-16 04:33:00+00:00                       2
                           groupInServiceCapacity
datetime                                         
2022-03-16 04:38:00+00:00                       3
2022-03-16 04:39:00+00:00                       3
                           groupInServiceCapacity
datetime                                         
2022-03-16 04:45:00+00:00                       1
2022-03-16 04:46:00+00:00                       1
2022-03-16 04:47:00+00:00                       1
2022-03-16 04:48:00+00:00                       1
2022-03-16 04:49:00+00:00                       1
2022-03-16 04:50:00+00:00                       2


                           groupInServiceCapacity
datetime                                         
2022-03-15 10:28:00+00:00                       1
2022-03-15 10:29:00+00:00                       1
2022-03-15 10:30:00+00:00                       1
2022-03-15 10:31:00+00:00                       1
2022-03-15 10:32:00+00:00                       2
2022-03-15 10:33:00+00:00                       2
                           groupInServiceCapacity
datetime                                         
2022-03-15 10:38:00+00:00                       3
2022-03-15 10:39:00+00:00                       3
                           groupInServiceCapacity
datetime                                         
2022-03-15 10:45:00+00:00                       1
2022-03-15 10:46:00+00:00                       1
2022-03-15 10:47:00+00:00                       1
2022-03-15 10:48:00+00:00                       1
2022-03-15 10:49:00+00:00                       1
2022-03-15 10:50:00+00:00                       2
